# Import Required Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
#!pip install catboost
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
%matplotlib inline
# ignore the warnings
import warnings
warnings.filterwarnings("ignore")
sns.set(rc={'figure.figsize': [7, 7]}, font_scale=1.2)
%config Completer.use_jedi=False
pd.set_option("display.max_columns", None)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read Train and Test Data

In [ ]:
dataset_path = '/kaggle/input/bike-sharing-demand/'

dfTrainO = pd.read_csv(os.path.join(dataset_path, 'train.csv'))
dfTestO = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
print("The shape of the dataset is {}.\n\n".format(dfTrainO.shape))
print("The shape of the dataset is {}.\n\n".format(dfTestO.shape))
dfTrainO.head()

In [ ]:
# dfTrainO=pd.read_csv("train.csv")
# print('---'*30)
# print('Train Head')
# print('---'*30)
# display(dfTrainO.sample(5))
# dfTestO=pd.read_csv("test.csv")
# print('---'*30)
# print('Test Head')
# print('---'*30)
# display(dfTestO.sample(5))
# # dfTestTrueY=pd.read_csv("yTest.csv")
# # dfTestTrueY.drop(columns='Unnamed: 0',inplace=True)
# # dfTestTrueY['WorkingDay'] = np.where((dfTestTrueY['WorkingDay']==1 )& (dfTestTrueY['Holiday']=="Holiday"), 0, dfTestTrueY['WorkingDay'])
# # print('---'*30)
# # print('Teat True Head')
# # print('---'*30)
# # display(dfTestTrueY.head())

__________________________________________________

# DateTime Handel

### Train

In [ ]:
dictYears={2011:0,2012:1}
##############################################################################
dfTrainO["Date"]            = pd.to_datetime(dfTrainO["datetime"], format='%Y/%m/%d')
dfTrainO["Year"]            = dfTrainO['Date'].dt.year
dfTrainO["Month"]           = dfTrainO['Date'].dt.month
dfTrainO["Day"]             = dfTrainO["Date"].dt.day
dfTrainO["Hour"]             = dfTrainO["Date"].dt.hour
dfTrainO["DayName"]         = dfTrainO['Date'].dt.day_name()
dfTrainO["DayNumber"]       = dfTrainO["Date"].dt.dayofweek
dfTrainO["Year"]            = dfTrainO["Year"].map(dictYears)
dfTrainO.drop(columns='Date',inplace=True)

In [ ]:
dfTrainO.head()

### Test

In [ ]:
dfTestO["Date"]            = pd.to_datetime(dfTestO["datetime"], format='%Y/%m/%d')
dfTestO["Year"]            = dfTestO['Date'].dt.year
dfTestO["Month"]           = dfTestO['Date'].dt.month
dfTestO["Day"]             = dfTestO["Date"].dt.day
dfTestO["Hour"]             = dfTestO["Date"].dt.hour
dfTestO["DayName"]         = dfTestO['Date'].dt.day_name()
dfTestO["DayNumber"]       = dfTestO["Date"].dt.dayofweek
dfTestO["Year"]            = dfTestO["Year"].map(dictYears)
dfTestO.drop(columns='Date',inplace=True)

In [ ]:
dfTestO.head()

# Rename Columns

### Train

In [ ]:
dfTrainO.columns

In [ ]:
dfTrainO.rename(columns={"season": 'Seasons',"holiday": 'Holiday','humidity':'Humidity','windspeed':'Wind_Speed','weather':'Weather','temp':'Temperature','atemp':'aTemperature','casual':'Casual','registered':'Registered','workingday':'WorkingDay'},inplace=True)

In [ ]:
dfTrainO.columns

In [ ]:
dfTrainO.head()

### Test

In [ ]:
dfTestO.columns

In [ ]:
dfTestO.rename(columns={"season": 'Seasons',"holiday": 'Holiday','humidity':'Humidity','windspeed':'Wind_Speed','weather':'Weather','temp':'Temperature','atemp':'aTemperature','workingday':'WorkingDay'},inplace=True)

In [ ]:
dfTestO.columns

In [ ]:
dfTestO.head()

# Rearrange the Columns

### Train

In [ ]:
dfTrainO = dfTrainO[['datetime','Year', 'Month','Day', 'DayName',
       'DayNumber','Hour', 'Weather','Temperature','count',
       'Humidity', 'Wind_Speed','Seasons', 'Holiday',
       'WorkingDay','Casual','Registered']]

In [ ]:
dfTrainO.head()

### Test

In [ ]:
dfTestO = dfTestO[['datetime','Year', 'Month','Day', 'DayName',
       'DayNumber','Hour', 'Weather','Temperature', 'Humidity', 'Wind_Speed',
       'Seasons', 'Holiday','WorkingDay']]

In [ ]:
dfTestO.head()

## Check The Shape

In [ ]:
dfTrainO.shape

In [ ]:
dfTestO.shape #(The difference is the Y Target)

# Edit the Temperature to be Integer

#### Train

In [ ]:
dfTrainO['Temperature']=np.floor(dfTrainO['Temperature']).astype(int)

In [ ]:
dfTrainO.head()

#### Test

In [ ]:
dfTestO['Temperature']=np.floor(dfTestO['Temperature']).astype(int)

In [ ]:
dfTestO.head()

# Create a Copy of the Data Frames to work with

In [ ]:
dfTrainNEW=dfTrainO.copy()

In [ ]:
dfTrainNEW.info()

In [ ]:
dfTestNEW=dfTestO.copy()

In [ ]:
dfTestNEW.info()

# Feature Engineering to the Date and Hour (Sin & Cos) DayName (Sin & Cos)

In [ ]:
dfTrainNEW.groupby('Hour')['count'].sum().plot(kind='bar');

In [ ]:
dfTrainNEW.groupby('Seasons')['count'].sum().plot(kind='bar');

In [ ]:
def extract_period_of_day1(hour):
    if hour in range(12):
        return 'Morning'
    elif hour in range(12, 18):
        return 'Afternoon'
    elif hour in range(18, 22):
        return 'Evening'
    else:
        return 'Night'
def extract_period_of_day2(hour):
    if hour in range(6,19):
        return 1
    elif hour in range(19,24):
        return 0
    else:
        return 0
def is_rush_hour(hour):
    return 0 if hour in [0,1,2,3,4,5,6,10,21,22,23] else 1

## Train

In [ ]:
dfTrainNEW['Hour_Sin']    = np.sin(dfTrainNEW['Hour']*(2.*np.pi/24))
dfTrainNEW['Hour_Cos']    = np.cos(dfTrainNEW['Hour']*(2.*np.pi/24))
dfTrainNEW['DayName_Sin'] = np.sin(pd.Categorical(dfTrainNEW['DayName']).codes*(2.*np.pi/7))
dfTrainNEW['DayName_Cos'] = np.cos(pd.Categorical(dfTrainNEW['DayName']).codes*(2.*np.pi/7))

In [ ]:
dfTrainNEW['Day_Period'] = dfTrainNEW['Hour'].apply(extract_period_of_day1)
dfTrainNEW['Rush_Hour']  = dfTrainNEW['Hour'].apply(is_rush_hour)
dfTrainNEW.head()

In [ ]:
dfTrainNEW.info()

## Test

In [ ]:
dfTestNEW['Hour_Sin']    = np.sin(dfTestNEW['Hour']*(2.*np.pi/24))
dfTestNEW['Hour_Cos']    = np.cos(dfTestNEW['Hour']*(2.*np.pi/24))
dfTestNEW['DayName_Sin'] = np.sin(pd.Categorical(dfTestNEW['DayName']).codes*(2.*np.pi/7))
dfTestNEW['DayName_Cos'] = np.cos(pd.Categorical(dfTestNEW['DayName']).codes*(2.*np.pi/7))

In [ ]:
dfTestNEW['Day_Period'] = dfTestNEW['Hour'].apply(extract_period_of_day1)
dfTestNEW['Rush_Hour']  = dfTestNEW['Hour'].apply(is_rush_hour)
dfTestNEW.head()

In [ ]:
dfTestNEW.info()

In [ ]:
print('dfTrainNew Columns:\n',dfTrainNEW.columns.tolist())
print('----'*50)
print('dfTestNew Columns:\n',dfTestNEW.columns.tolist())
print('----'*50)
#print('dfTestTrueY Columns:\n',dfTestTrueY.columns.tolist())

### Check

In [ ]:
dfTrainNEW['Day_Period'].value_counts()

In [ ]:
dfTestNEW['Day_Period'].value_counts()

In [ ]:
dfTrainNEW["Rush_Hour"].value_counts()

In [ ]:
dfTestNEW["Rush_Hour"].value_counts()

# Value Counts and Uniques Equations

In [ ]:
def ValueCounts(df):
    for c in df.columns:
        print(c+"\n"+"-----------------"+"\n")
        print(df[c].value_counts().to_frame())
        print("\n"+"******************"+"\n")

In [ ]:
def UniqueValues(df,l):
    for c in l:
        print(c+"\n"+"-----------------"+"\n")
        print(df[c].unique())
        print("\n"+"******************"+"\n")

In [ ]:
ValueCounts(dfTrainNEW)

In [ ]:
ValueCounts(dfTestNEW)

In [ ]:
UniqueValues(dfTrainNEW,dfTrainNEW.columns)

In [ ]:
UniqueValues(dfTestNEW,dfTestNEW.columns)

In [ ]:
dfTrainNEW.describe().T

In [ ]:
dfTestNEW.describe().T

In [ ]:
# dfTestTrueY.describe().T

_________________________________________________

In [ ]:
# dfTestTrueY.groupby('Hour')['y'].sum().plot(kind='bar');

In [ ]:
# dfTestTrueY.groupby('Seasons')['y'].sum().plot(kind='bar');

_______________________________________________________

# Check for skewness

## Train

In [ ]:
dfTrainNEW.columns

In [ ]:
figure, axes = plt.subplots(nrows=3, ncols=3)
figure.set_size_inches(20,15)
figure.tight_layout()
sns.kdeplot(dfTrainNEW['Temperature']    ,shade=True, ax=axes[0][0]);
sns.kdeplot(dfTrainNEW['count']          ,shade=True, ax=axes[0][1]);
sns.kdeplot(dfTrainNEW['Humidity']       ,shade=True, ax=axes[0][2]);
sns.kdeplot(dfTrainNEW['Wind_Speed']     ,shade=True, ax=axes[1][0]);
sns.kdeplot(dfTrainNEW['Weather']        ,shade=True, ax=axes[1][1]);
sns.kdeplot(dfTrainNEW['Hour']           ,shade=True, ax=axes[1][2]);
sns.kdeplot(dfTrainNEW['Seasons']        ,shade=True, ax=axes[2][0]);
sns.kdeplot(dfTrainNEW['Registered']     ,shade=True, ax=axes[2][1]);
sns.kdeplot(dfTrainNEW['Casual']         ,shade=True, ax=axes[2][2]);

## True Test

In [ ]:
figure, axes = plt.subplots(nrows=2, ncols=3)
figure.set_size_inches(20,15)
figure.tight_layout()
sns.kdeplot(dfTestNEW['Temperature']    ,shade=True, ax=axes[0][0]);
sns.kdeplot(dfTestNEW['Humidity']       ,shade=True, ax=axes[0][1]);
sns.kdeplot(dfTestNEW['Wind_Speed']     ,shade=True, ax=axes[0][2]);
sns.kdeplot(dfTestNEW['Weather']        ,shade=True, ax=axes[1][0]);
sns.kdeplot(dfTestNEW['Hour']           ,shade=True, ax=axes[1][1]);
sns.kdeplot(dfTestNEW['Seasons']        ,shade=True, ax=axes[1][2]);

# Features to Check: `Count`, `Wind_Speed`

## `count`

### Train

In [ ]:
dfTrainNEW['logcount']   =dfTrainNEW['count'].apply(np.log1p)
dfTrainNEW['sqrtcount']  =dfTrainNEW['count'].apply(np.sqrt)
dfTrainNEW['squarcount'] =np.power(dfTrainNEW['count'],2)

In [ ]:
figure, axes = plt.subplots(nrows=1, ncols=4)
figure.set_size_inches(20,10)
sns.kdeplot(dfTrainNEW['count']      ,shade=True, ax=axes[0]);
sns.kdeplot(dfTrainNEW['logcount']   ,shade=True, ax=axes[1]);
sns.kdeplot(dfTrainNEW['sqrtcount']  ,shade=True, ax=axes[2]);
sns.kdeplot(dfTrainNEW['squarcount'] ,shade=True, ax=axes[3]);

## `Wind_Speed`

### Train

In [ ]:
dfTrainNEW['logWind_Speed']   =dfTrainNEW['Wind_Speed'].apply(np.log1p)
dfTrainNEW['sqrtWind_Speed']  =dfTrainNEW['Wind_Speed'].apply(np.sqrt)
dfTrainNEW['squarWind_Speed'] =np.power(dfTrainNEW['Wind_Speed'],2)

In [ ]:
figure, axes = plt.subplots(nrows=1, ncols=4)
figure.set_size_inches(20,10)
sns.kdeplot(dfTrainNEW['Wind_Speed']      ,shade=True, ax=axes[0]);
sns.kdeplot(dfTrainNEW['logWind_Speed']   ,shade=True, ax=axes[1]);
sns.kdeplot(dfTrainNEW['sqrtWind_Speed']  ,shade=True, ax=axes[2]);
sns.kdeplot(dfTrainNEW['squarWind_Speed'] ,shade=True, ax=axes[3]);

# `SquareRoot` Y        ------> Best

# Drop the Columns we Don't Need

In [ ]:
dfTrainNEW.columns

In [ ]:
dfTrainNEW.drop(columns=['logcount','squarcount','sqrtWind_Speed','logWind_Speed','squarWind_Speed'],inplace=True)

In [ ]:
dfTrainNEW.info()

In [ ]:
dfTestNEW.info()

________________________________________________

# Plots To Visualize the Data

## Check

In [ ]:
set(dfTrainNEW["Seasons"])

In [ ]:
# print('Train')
# sns.pairplot(dfTrainNEW)

# print('Test True')
# sns.pairplot(dfTestNEW)

# Correlations

## Train

In [ ]:
corr_matrix = dfTrainNEW.corr()
mask = np.array(corr_matrix)
mask[np.tril_indices_from(mask)] = False

fig = plt.figure(figsize=(25, 20))
sns.heatmap(np.round(corr_matrix,2), mask=mask, annot=True, cbar=True, vmax=0.8, vmin=-0.8, cmap='RdYlGn')
plt.show()

In [ ]:
abs(dfTrainNEW.corr()["count"]).sort_values(ascending=False)

## Outliers Check

In [ ]:
def zscore(series): 
    return (series-series.mean())/series.std()
dfTrainNEW_O=dfTrainNEW.copy()
dfTrainNEW_O['count_zscore'] = dfTrainNEW.groupby(['Hour', 'WorkingDay'])['count'].transform(zscore)
outlier_idx = np.abs(dfTrainNEW_O['count_zscore'])>3
outlier_data = dfTrainNEW_O.loc[outlier_idx, :]
print('Shape of the outlier data entries: ', outlier_data.shape)
outlier_data

##### Removing Outliers from Data

In [ ]:
#Removing outliers from Data
dfTrainNEW_Wo = dfTrainNEW_O.loc[~outlier_idx, :]
print('Shape of Data Before Outlier Pruning: ', dfTrainNEW.shape)
print('Shape of Data After Outlier Pruning: ', dfTrainNEW_Wo.shape)

In [ ]:
# dfTrainNEW.drop(['Hour', 'DayName'], axis=1, inplace=True)
# dfTestNEW.drop(['Hour', 'DayName'], axis=1, inplace=True)

In [ ]:
dfTrainNEW.head()

In [ ]:
dfTestNEW.head()

# Categorical Columns Encoding

## 1. One Hot Encoding

In [ ]:
dfTrainNEW.info()

In [ ]:
dfTrainEncoded_Dummies =dfTrainNEW.copy()
dfTestEncoded_Dummies  =dfTestNEW.copy()

In [ ]:
dfTrainEncoded_Dummies = pd.get_dummies(dfTrainEncoded_Dummies,columns=['Day_Period'],drop_first=True)
dfTestEncoded_Dummies  = pd.get_dummies(dfTestEncoded_Dummies, columns=['Day_Period'],drop_first=True)

##### Train

In [ ]:
dfTrainEncoded_Dummies.head()

In [ ]:
dfTrainEncoded_Dummies.info()

###### Test

In [ ]:
dfTestEncoded_Dummies.head()

In [ ]:
dfTestEncoded_Dummies.info()

_______________________________

## 2. Label Encoding

In [ ]:
dfTrainEncoded_Labels =dfTrainNEW.copy()
dfTestEncoded_Labels  =dfTestNEW.copy()

In [ ]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
dfTrainEncoded_Labels=MultiColumnLabelEncoder(columns = ['Day_Period']).fit_transform(dfTrainEncoded_Labels)
dfTestEncoded_Labels=MultiColumnLabelEncoder(columns =  ['Day_Period']).fit_transform(dfTestEncoded_Labels)

##### Train

In [ ]:
dfTrainEncoded_Labels.head()

In [ ]:
dfTrainEncoded_Labels.info()

###### Test

In [ ]:
dfTestEncoded_Labels.head()

In [ ]:
dfTestEncoded_Labels.info()

________________

# Check the Correlation

## 1. One Hot Encoding

In [ ]:
abs(dfTrainEncoded_Dummies.corr()["count"]).sort_values()

In [ ]:
abs(dfTrainEncoded_Dummies.corr()["count"]).sort_values().index

In [ ]:
dfTrainEncoded_Dummies.columns

## 2. Label Encoding

In [ ]:
abs(dfTrainEncoded_Labels.corr()["count"]).sort_values()

In [ ]:
abs(dfTrainEncoded_Labels.corr()["count"]).sort_values().index

____________________________________________

# Machine Learning Model 

## Train Data

********************************************
********************************************
********************************************

In [ ]:
dfTrain = dfTrainEncoded_Dummies.drop(columns = ['Casual','Registered','sqrtcount','DayName'])#hour, snowfall, windspeed
# dfTrain.info()

In [ ]:
dfTest  = dfTestEncoded_Dummies.drop(columns = ['DayName'])
# dfTest.info()

In [ ]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(dfTrain, test_size=0.2, random_state=42,stratify =None)
X_train = train_df.drop(columns=["datetime","count"])
y_train = train_df['count']
################################################################################################
X_val = val_df.drop(columns=["datetime","count"])
y_val = val_df['count']

In [ ]:
y_trainLog=np.log1p(y_train)
y_valLog=np.log1p(y_val)

In [ ]:
# X_train=normalize(X_train)
# X_val=normalize(X_val)
# dfTest=normalize(dfTest)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_val = scaler.transform(X_val)

In [ ]:
# from sklearn.preprocessing import minmax_scale
# X_train = minmax_scale(X_train, feature_range=(0, 1))
# X_val = minmax_scale(X_val, feature_range=(0, 1))

In [ ]:
# from sklearn.preprocessing import RobustScaler
# rb = RobustScaler()
# X_train= rb.fit_transform(X_train)
# X_val = rb.fit_transform(X_val)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler  = StandardScaler()
X_train = scaler.fit_transform(X_train.values)
X_val   = scaler.fit_transform(X_val.values)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import BaggingRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score

In [ ]:
def rmsle(y_true, y_pred, convertExp=True):
    # Apply exponential transformation function
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
        
    # Convert missing value to zero after log transformation
    log_true = np.nan_to_num(np.array([np.log(y+1) for y in y_true]))
    log_pred = np.nan_to_num(np.array([np.log(y+1) for y in y_pred]))
    
    # Compute RMSLE
    output = np.sqrt(np.mean((log_true - log_pred)**2))
    return output

In [ ]:
models = {
    "LinearRegression":            LinearRegression(),
    "Ridge":                       Ridge(),
    "Lasso":                       Lasso(),
    "ElasticNet":                  ElasticNet(),
    "KNeighborsRegressor":         KNeighborsRegressor(), 
    "DecisionTreeRegressor":       DecisionTreeRegressor(),
    "RandomForestRegressor":       RandomForestRegressor(),
    "BaggingRegressor":            BaggingRegressor(),
    "AdaBoostRegressor":           AdaBoostRegressor(),
    "CatBoostRegressor":           CatBoostRegressor(verbose=False),
    "LGBMRegressor":               LGBMRegressor(),
    "GradientBoostingRegressor":   GradientBoostingRegressor(),
    "XGBRegressor":                XGBRegressor()
}

In [ ]:
for name, model in models.items():
    print(f'Using model: {name}')
    model.fit(X_train, y_trainLog)
    print(f"train score : {model.score(X_train,y_trainLog)}")
    print(f'RMSLETrain: {rmsle(y_trainLog, model.predict(X_train))}')
    print(f'RMSLEVal: {rmsle(y_valLog, model.predict(X_val))}')
    print('-'*30)

_____________________________________________________________

## Test Data

**************************************
**************************************
**************************************

In [ ]:
datetime=dfTest["datetime"]
X_test = dfTest.drop(columns=['datetime'])
X_test=scaler.transform(X_test)

In [ ]:
# Model=XGBRegressor("reg:squaredlogerror")
# Model.fit(X_train, y_trainLog)

# print(f'RMSLETrain: {rmsle(y_trainLog, Model.predict(X_train))}')
# print(f'RMSLEVal: {rmsle(y_valLog, Model.predict(X_val))}')
# print("-"*10)

In [ ]:
# model = CatBoostRegressor(verbose=False)
# parameters = {'depth'          : [1,2,3,4,5,6],
#               'learning_rate'  : [0.01, 0.05, 0.1,0.15,0.2,0.25],
#               'n_estimators'   : [100,200,300,400,500,600],
#               'l2_leaf_reg'    : [1,2,3,4,5,6],
#               'random_strength': [0.6,0.61,0.62,0.65,0.665,0.7]
#               }

# grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 4, n_jobs=-1)
# grid.fit(X_train, y_trainLog)

In [ ]:
Model=CatBoostRegressor(verbose=False, n_estimators=500 ,learning_rate=0.15,depth=3,l2_leaf_reg=2,random_strength=0.665)
Model.fit(X_train, y_trainLog)

print(f'RMSLETrain: {rmsle(y_trainLog, Model.predict(X_train))}')
print(f'RMSLEVal: {rmsle(y_valLog, Model.predict(X_val))}')


In [ ]:
y_test_predicted = Model.predict(X_test)

dfTest['count'] = np.floor(np.exp(y_test_predicted)).astype('int') 
yPredictTest=dfTest['count']

In [ ]:
dfTest.drop(columns="datetime",inplace=True)
dfTest=pd.concat([dfTest, datetime],axis=1)
dfTest.head()

In [ ]:
dfTest[['datetime', 'count']].to_csv('/kaggle/working/submission.csv', index=False)
# dfTest[['datetime', 'count']].to_csv('submission.csv', index=False)

________________________________________________________